# Creating artist table in SQL database

To aid my SQL Spotify Wrapped project I need an artist table in my database. This will contain info about the artist (genre, popularity, etc) and can be joined to my (streaming) history table on the artist name. (Or I might have to add artist IDs).

To populate the artist table I will use the Spotify API and spotipy in python.

Below I also take the opportunity to delete podcasts from the history table. They show up as most listened by msPlayed and I'm not interested in this. 

### Set up

In [1]:
import config as cfg

# Spotify stuff
import spotipy  # this is a python wrapper for interacting with the Spotify API
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=cfg.client_id, client_secret=cfg.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# SQL
import sqlite3

# pandas just for displaying nice table
import pandas as pd


### Get artist info

How to get info about a specific artist

In [2]:
artist_name = 'Kanye West' # for example
artist_uri = sp.search(q ='artist:' + artist_name, type='artist')['artists']['items'][0]['uri']
print(artist_name + ' genres:', sp.artist(artist_uri)['genres'])
print(artist_name + ' popularity:', sp.artist(artist_uri)['popularity'])

Kanye West genres: ['chicago rap', 'rap']
Kanye West popularity: 95


### Create new table

Loop though artists in history table. Search artists using spotipy. Fill in new table.

In [2]:
# connect to database and create cursor object
con = sqlite3.connect('db.sqlite3')
cur = con.cursor()

In [4]:
# query distinct arists in history table
sql_command = """SELECT DISTINCT artistName FROM history"""
cur.execute(sql_command)
artist_names = cur.fetchall()
N_artists = len(artist_names)
print('This year I have listened to', N_artists, 'different artists')
print('The first 20 from the list are:')
print(artist_names[0:20]) 

This year I have listened to 1125 different artists
The first 20 from the list are:
[('Whitney Houston',), ('Stereophonics',), ('King Krule',), ('Ariana Grande',), ('Todd Terry',), ('ZAYN',), ('ON / OFF / ON',), ('Night Flight',), ('Jammz',), ('Filthy Gears',), ('Meridian Dan',), ('Ghetts',), ('Sox',), ('Nick Brewer',), ('President T',), ('Bugzy Malone',), ('Big Narstie',), ('Halo',), ('P Money',), ('Oscar #Worldpeace',)]


In [88]:
# create new table. THIS TAKES A LONG TIME TO RUN!
sql_command = """
CREATE TABLE IF NOT EXISTS [artist] (
[artistName] VARCHAR NULL UNIQUE,
[genre] VARCHAR NULL,
[popularity] INT NULL
);
"""
cur.execute(sql_command)

# fill table
# This takes a while to run. It may be more efficient to create one large SQL command and run that in one go rather than repeatedly running small commands. 
cur.execute("""BEGIN TRANSACTION""")

for row in artist_names:
    artist_name = row[0]
    
    # find artist
    try:
        artist_uri = sp.search(q ='artist:' + artist_name, type='artist')['artists']['items'][0]['uri']
    except: 
        continue  # if no artist found, skip to next. 
    
    # find genre
    try:
        genre = sp.artist(artist_uri)['genres'][0]
    except:
        genre = 'NULL'  # if no genre found
    
    # find popularity
    popularity = sp.artist(artist_uri)['popularity']
    
    sql_command = """INSERT INTO artist VALUES (?,?,?)"""# put data in here
    cur.execute(sql_command, [artist_name, genre, popularity])

cur.execute("""COMMIT""")


#  Deleteing podcasts from history

In [3]:
# delete podcasts I listen too.
cur.execute("""DELETE FROM history WHERE artistName = "Dan Carlin's Hardcore History" """)
cur.execute("""DELETE FROM history WHERE artistName = "F1: Beyond The Grid" """)
cur.execute("""DELETE FROM history WHERE artistName = "THE ADAM BUXTON PODCAST" """)
cur.execute("""DELETE FROM history WHERE artistName = "Athletico Mince" """)
cur.execute("""DELETE FROM history WHERE artistName = "Desert Island Discs" """)
cur.execute("""DELETE FROM history WHERE artistName = "Siraj Raval Podcast" """)
cur.execute("""DELETE FROM history WHERE artistName = "Getting Curious with Jonathan Van Ness" """)
cur.execute("""DELETE FROM history WHERE artistName = "Reasons to be Cheerful with Ed Miliband and Geoff Lloyd" """)
cur.execute("""DELETE FROM history WHERE artistName = "Cheerful Election Daily with Owen Jones" """)
cur.execute("""DELETE FROM history WHERE artistName = "Today in Focus" """)

# check deleted
cur.execute("""SELECT * FROM history WHERE artistName = "Today in Focus" """)
print(cur.fetchmany(10))


[]


In [4]:
con.commit()
con.close()